In [1]:
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

import numpy as np
from emo_utils import *
import matplotlib.pyplot as plt

%matplotlib inline

In [2]:
X_train, Y_train = read_csv('../data/news_news_small.outfinal')
filtered = list(filter(None, Y_train))
Y_train = np.asarray(filtered)
Y_train = Y_train.astype(int)
#print(Y_train) make sure theyre ints
X_test, Y_test = read_csv('../data/news_news_small_test.outfinal')
filtered = list(filter(None, Y_test))
Y_test = np.asarray(filtered)
Y_test = Y_test.astype(int)
#print(Y_test) make sure theyre ints

Y_train = np.append(Y_train, [0])
print(X_train.shape, Y_train.shape)
Y_test = np.append(Y_test, [0])
print(X_test.shape, Y_test.shape)


(942,) (942,)
(636,) (636,)


In [3]:
def read_glove_vecs(glove_file):
    with open(glove_file, 'r') as f:
        words = set()
        word_to_vec_map = {}
        for line in f:
            line = line.strip().split()
            curr_word = line[0]
            words.add(curr_word)
            word_to_vec_map[curr_word] = np.array(line[1:], dtype=np.float64)
        
        i = 1
        words_to_index = {}
        index_to_words = {}
        for w in sorted(words):
            words_to_index[w] = i
            index_to_words[i] = w
            i = i + 1
    return words_to_index, index_to_words, word_to_vec_map

In [4]:
word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('../data/glove.6B.50d.txt')

In [5]:
word = "hollow"
index = 232536
print("the index of", word, "in the vocabulary is", word_to_index[word])
print("the", str(index) + "th word in the vocabulary is", index_to_word[index])
print(word_to_index['word'])
print(len(X_train))

the index of hollow in the vocabulary is 180658
the 232536th word in the vocabulary is man
389793
942


In [21]:
def remove_non_keys(X, dict):
#     for line in X:
#         for word in line.split(" "):
#             if (not (word in dict)):
#                 if (not (word == " ")):
#                     if (not (word == "")):
#                         original_line = line
# #                         print(original_line)
#                         replacement = line.replace(word, '')
#                         replacement = " ".join(replacement.split())
#                         print(X[list(X).index(original_line)])
#                         X[list(X).index(original_line)] = replacement
#                         X[list(X).index(line)].replace(word, '')
#                         print
#                         print(list(X).index(line))
#                 X[list(X).index(line)].replace(word, '')               
#     for line in X:
#         for word in line.split(" "):
#             if not (word in dict):
#                 print(word)
    for i in range(0, len(X)):
        for word in X[i].split(" "):
            if (not (word in dict)):
                if (not (word == " ")):
                    if (not (word == "")):
                        X[i] = X[i].replace(word, '')
                        X[i] = " ".join(X[i].split())
    return X
                        

In [22]:
X_train = remove_non_keys(X_train, word_to_index)
X_test = remove_non_keys(X_test, word_to_index)

for i in range(0, len(X_train)):
    for word in X_train[i].split(" "):
        if (not (word in word_to_vec_map)):
            print(word)
            if (not (word == " ")):
                if (not (word == "")):
                    print(word)
                    

In [23]:

# GRADED FUNCTION: sentence_to_avg

def sentence_to_avg(sentence, word_to_vec_map):
    """
    Converts a sentence (string) into a list of words (strings). Extracts the GloVe representation of each word
    and averages its value into a single vector encoding the meaning of the sentence.
    
    Arguments:
    sentence -- string, one training example from X
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    
    Returns:
    avg -- average vector encoding information about the sentence, numpy-array of shape (50,)
    """
    
    ### START CODE HERE ###
    # Step 1: Split sentence into list of lower case words (≈ 1 line)
    words = [i.lower() for i in sentence.split()]

    # Initialize the average word vector, should have the same shape as your word vectors.
    avg = np.zeros((50,))
    
    # Step 2: average the word vectors. You can loop over the words in the list "words".
    for w in words:
        avg += word_to_vec_map[w]
    avg = avg / len(words)
    
    ### END CODE HERE ###
    
    return avg

In [24]:
avg = sentence_to_avg("Morrocan couscous is my favorite dish", word_to_vec_map)
print("avg = ", avg)

avg =  [-0.008005    0.56370833 -0.50427333  0.258865    0.55131103  0.03104983
 -0.21013718  0.16893933 -0.09590267  0.141784   -0.15708967  0.18525867
  0.6495785   0.38371117  0.21102167  0.11301667  0.02613967  0.26037767
  0.05820667 -0.01578167 -0.12078833 -0.02471267  0.4128455   0.5152061
  0.38756167 -0.898661   -0.535145    0.33501167  0.68806933 -0.2156265
  1.797155    0.10476933 -0.36775333  0.750785    0.10282583  0.348925
 -0.27262833  0.66768    -0.10706167 -0.283635    0.59580117  0.28747333
 -0.3366635   0.23393817  0.34349183  0.178405    0.1166155  -0.076433
  0.1445417   0.09808667]


In [25]:
# GRADED FUNCTION: model

def model(X, Y, word_to_vec_map, learning_rate = 0.01, num_iterations = 400):
    """
    Model to train word vector representations in numpy.
    
    Arguments:
    X -- input data, numpy array of sentences as strings, of shape (m, 1)
    Y -- labels, numpy array of integers between 0 and 7, numpy-array of shape (m, 1)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    learning_rate -- learning_rate for the stochastic gradient descent algorithm
    num_iterations -- number of iterations
    
    Returns:
    pred -- vector of predictions, numpy-array of shape (m, 1)
    W -- weight matrix of the softmax layer, of shape (n_y, n_h)
    b -- bias of the softmax layer, of shape (n_y,)
    """
    
    np.random.seed(1)

    # Define number of training examples
    m = Y.shape[0]                          # number of training examples
    n_y = 10                                 # number of classes  
    n_h = 50                                # dimensions of the GloVe vectors 
    
    # Initialize parameters using Xavier initialization
    W = np.random.randn(n_y, n_h) / np.sqrt(n_h)
    b = np.zeros((n_y,))
    
    # Convert Y to Y_onehot with n_y classes
    Y_oh = convert_to_one_hot(Y, C = n_y) 
    
    # Optimization loop
    for t in range(num_iterations):                       # Loop over the number of iterations
        for i in range(m):                                # Loop over the training examples
            
            ### START CODE HERE ### (≈ 4 lines of code)
            # Average the word vectors of the words from the i'th training example
            avg = sentence_to_avg(X[i], word_to_vec_map)

            # Forward propagate the avg through the softmax layer
            z = np.dot(W, avg) + b
            a = softmax(z)

            # Compute cost using the i'th training label's one hot representation and "A" (the output of the softmax)
            cost = -np.sum(np.multiply(Y_oh[i], np.log(a)))
            ### END CODE HERE ###
            
            # Compute gradients 
            dz = a - Y_oh[i]
            dW = np.dot(dz.reshape(n_y,1), avg.reshape(1, n_h))
            db = dz

            # Update parameters with Stochastic Gradient Descent
            W = W - learning_rate * dW
            b = b - learning_rate * db
        
        if t % 100 == 0:
            print("Epoch: " + str(t) + " --- cost = " + str(cost))
            pred = predict(X, Y, W, b, word_to_vec_map)

    return pred, W, b

In [26]:
print(X_train.shape)
print(Y_train.shape)
print(np.eye(10)[Y_train.reshape(-1)].shape)
print(X_train[0])
print(type(X_train))
Y = np.asarray([5,0,0,5, 4, 4, 4, 6, 6, 4, 1, 1, 5, 6, 6, 3, 6, 3, 4, 4])
print(Y.shape)

X = np.asarray(['I am going to the bar tonight', 'I love you', 'miss you my dear',
 'Lets go party and drinks','Congrats on the new job','Congratulations',
 'I am so happy for you', 'Why are you feeling bad', 'What is wrong with you',
 'You totally deserve this prize', 'Let us go play football',
 'Are you down for football this afternoon', 'Work hard play harder',
 'It is suprising how people can be dumb sometimes',
 'I am very disappointed','It is the best day in my life',
 'I think I will end up alone','My life is so boring','Good job',
 'Great so awesome'])

print(X.shape)
print(np.eye(10)[Y_train.reshape(-1)].shape)
print(type(X_train))

(942,)
(942,)
(942, 10)
i played football for 4 years in highschool started all 4 years i will never allow my kids to play unless they really really want to id rather have them play a sport with less contact or have them do theater because thats just as fun
<class 'numpy.ndarray'>
(20,)
(20,)
(942, 10)
<class 'numpy.ndarray'>


In [27]:
pred, W, b = model(X_train, Y_train, word_to_vec_map)
print(pred)

Epoch: 0 --- cost = 2.866968183928237
Accuracy: 0.262208067940552
Epoch: 100 --- cost = 2.1322889096316406
Accuracy: 0.3248407643312102
Epoch: 200 --- cost = 2.15314394800427
Accuracy: 0.32590233545647557
Epoch: 300 --- cost = 2.206812390794382
Accuracy: 0.33014861995753714
[[1.]
 [6.]
 [1.]
 [9.]
 [1.]
 [9.]
 [6.]
 [9.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [6.]
 [1.]
 [1.]
 [6.]
 [5.]
 [1.]
 [1.]
 [1.]
 [1.]
 [9.]
 [9.]
 [1.]
 [5.]
 [1.]
 [6.]
 [5.]
 [1.]
 [9.]
 [1.]
 [4.]
 [0.]
 [9.]
 [1.]
 [6.]
 [1.]
 [1.]
 [1.]
 [1.]
 [9.]
 [1.]
 [6.]
 [6.]
 [1.]
 [1.]
 [1.]
 [6.]
 [1.]
 [1.]
 [6.]
 [8.]
 [1.]
 [1.]
 [1.]
 [1.]
 [8.]
 [1.]
 [1.]
 [1.]
 [1.]
 [6.]
 [1.]
 [6.]
 [6.]
 [1.]
 [6.]
 [3.]
 [9.]
 [1.]
 [6.]
 [9.]
 [6.]
 [9.]
 [9.]
 [1.]
 [6.]
 [6.]
 [6.]
 [6.]
 [1.]
 [1.]
 [1.]
 [9.]
 [1.]
 [9.]
 [1.]
 [6.]
 [3.]
 [6.]
 [9.]
 [1.]
 [1.]
 [1.]
 [1.]
 [6.]
 [1.]
 [0.]
 [0.]
 [6.]
 [1.]
 [0.]
 [0.]
 [0.]
 [1.]
 [6.]
 [1.]
 [1.]
 [7.]
 [1.]
 [6.]
 [8.]
 [9.]
 [6.]
 [1.]
 [1.]
 [7.]
 [7.]
 [1.]
 [9.]

In [28]:
print("Training set:")
pred_train = predict(X_train, Y_train, W, b, word_to_vec_map)
print('Test set:')
pred_test = predict(X_test, Y_test, W, b, word_to_vec_map)

Training set:
Accuracy: 0.32908704883227174
Test set:
Accuracy: 0.2468553459119497


In [29]:
X_my_sentences = np.array(["i adore you", "i love you", "funny lol", "lets play with a ball", "food is ready", "not feeling happy"])
Y_my_labels = np.array([[0], [0], [2], [1], [4],[3]])

pred = predict(X_my_sentences, Y_my_labels , W, b, word_to_vec_map)


Accuracy: 0.0


In [30]:
print_predictions(X_my_sentences, pred)

NameError: name 'emoji' is not defined